In [16]:
from keras.models import load_model

model = load_model('model/69.9_model.h5')


In [4]:
import glob # 재귀적으로 현재 폴더의 모든 하위폴더까지 탐색하여 확장자가 jpg인 파일을 출력한다.
            # 특정 파일만 출력하기
caltech_dir = "dataset/test/" # fire 데이터 폴더 경로


files = glob.glob(caltech_dir+"/*.jpg") # 이미지 파일 가져오기

for i, f in enumerate(files): # 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환
    view_img = cv2.imread(f, cv2.IMREAD_COLOR)
    view_img = cv2.resize(view_img,(200, 200))
    b, g, r = cv2.split(view_img) # img파일을 색 채널별 b,g,r로 분리
    data = cv2.merge([r,g,b])
    
    view_img = cv2.imread(f)
    face_extractor(view_img, cdir)

dataset/test\14.smiling-asian-woman-happy-selfie.leungchopan.Shutterstock-370x242.jpg


NameError: name 'cv2' is not defined

In [98]:
import cv2
import numpy as np

f= 'dataset/test/7.932775232.jpg'

view_img = cv2.imread(f)
face_extractor(view_img)

# model.prediect();

smile
smile
smile
smile


In [97]:


#전체 사진에서 얼굴 부위만 잘라 리턴
def face_extractor(img):
    face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    #얼굴 찾기 
    faces = face_classifier.detectMultiScale(img,1.3,5)
    #찾은 얼굴이 없으면 None으로 리턴 
    if faces is():
        return None
    #얼굴들이 있으면 
    for(x,y,w,h) in faces:
        #해당 얼굴 크기만큼 cropped_face에 잘라 넣기 
        #근데... 얼굴이 2개 이상 감지되면??
        #가장 마지막의 얼굴만 남을 듯
        cropped_face = img[y:y+h, x:x+w]
        face = cv2.resize(cropped_face,(48,48))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), thickness=2)
        label= pre(face)
        cv2.putText(img, label, (x+w, (y+h) - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 1)


    #cropped_face 리턴 

In [42]:
import cv2
import numpy as np

def pre(img):

    categories = ["neutral","angry","surprise","smile","sad"]  # 각 dataset 폴더이름
    img = img.reshape(-1,48,48,1)
    predict = model.predict(img)
    
    return categories[np.argmax(predict)]

# model.prediect();